In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold

from __future__ import absolute_import, division, print_function

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
import random
from tensorflow.keras import Model, layers

In [2]:
# df = pd.read_csv("test_classification_data_with_date.csv", index_col = False)
df = pd.read_csv("test_classification_data.csv", index_col = False)
# df = df.drop(df[(df.Class == 'H')].index).drop('date', axis = 1)
df = df.drop('date', axis = 1)
X = df.drop('Class', axis = 1)
Y = df['Class']

kf = KFold(n_splits = 3)

In [3]:
# Neural Network

# dataset parameters
num_classes = 2
num_features = 50

# Training parameters
learning_rate = 0.01
training_steps = 2000
batch_size = 256
display_steps = 250

# Network parameters
n_hidden_1 = 256
n_hidden_2 = 128

# Create NN model
class Neural_Network(Model):
    # Set layers
    def __init__(self):
        tf.keras.backend.clear_session()
        super(Neural_Network, self).__init__();
        # 1st fully connected hidden layer
        self.fc1 = layers.Dense(n_hidden_1, activation = tf.nn.relu)
        # 2nd fully connected hidden layer
        self.fc2 = layers.Dense(n_hidden_2, activation = tf.nn.relu)
        # Output layer
        self.out = layers.Dense(num_classes)
        
    # Set forward pass
    def call(self, x, is_training = False):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Cross-Entropy loss function.
def cross_entropy_NN(y_pred, y_true):
    # Convert labels to int64 for tf cross-entropy function
    y_true = tf.cast(y_true, tf.int64)
    # Apply softmax to logits and compute cross-entropy
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y_true, logits = y_pred)
    # Average loss across batch
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy_NN(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis = -1)

# Optimization process
def run_optimization_NN(x, y):
    with tf.GradientTape() as g:
        pred = neural_network(x, is_training = True)
        loss = cross_entropy_NN(pred, y)
        
    trainable_variables = neural_network.trainable_variables
    
    gradients = g.gradient(loss, trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [4]:
# Build NN model
neural_network = Neural_Network()

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

for train_index, test_index in kf.split(X):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
#     x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
#     x_train, x_test = x_train/255, x_test/255

    # Use tf.data API to shuffle and batch data.
    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.repeat().shuffle(20000).batch(batch_size).prefetch(1)
    for step, (batch_x, batch_y) in enumerate (train_data.take(training_steps), 1):
        run_optimization_NN(batch_x, batch_y)

        if step % display_steps == 0 or step == 5:
            pred = neural_network(batch_x, is_training = True)
            loss = cross_entropy_NN(pred, batch_y)
            acc = accuracy_NN(pred, batch_y)
            print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))
        
    # Test model on validation set.
    pred = neural_network(x_test, is_training = False)
    print("Test Accuracy: %f" % accuracy_NN(pred, y_test))

neural_network.save('test_NN_1.0')

step: 5, loss: 1.049319, accuracy: 0.535156
step: 250, loss: 0.079084, accuracy: 1.000000
step: 500, loss: 0.025989, accuracy: 1.000000
step: 750, loss: 0.016170, accuracy: 1.000000
step: 1000, loss: 0.010561, accuracy: 1.000000
step: 1250, loss: 0.008032, accuracy: 1.000000
step: 1500, loss: 0.004552, accuracy: 1.000000
step: 1750, loss: 0.004752, accuracy: 1.000000
step: 2000, loss: 0.003483, accuracy: 1.000000
Test Accuracy: 0.769231
step: 5, loss: 3.246548, accuracy: 0.433594
step: 250, loss: 0.007425, accuracy: 1.000000
step: 500, loss: 0.004235, accuracy: 1.000000
step: 750, loss: 0.003088, accuracy: 1.000000
step: 1000, loss: 0.002291, accuracy: 1.000000
step: 1250, loss: 0.001644, accuracy: 1.000000
step: 1500, loss: 0.001691, accuracy: 1.000000
step: 1750, loss: 0.001419, accuracy: 1.000000
step: 2000, loss: 0.001269, accuracy: 1.000000
Test Accuracy: 0.615385
step: 5, loss: 1.590968, accuracy: 0.644531
step: 250, loss: 0.012302, accuracy: 1.000000
step: 500, loss: 0.007025, a

In [9]:
x = np.array(X, np.float32)
pred = neural_network(x, is_training = False)
print(accuracy_NN(pred, Y))

T = pd.read_csv("test_data.csv", index_col = False)
T = T.drop('Date', axis = 1)
x_t = T.drop('Class', axis = 1)
x_t = np.array(x_t, np.float32)
y_t = T['Class']
pred = neural_network(x_t, is_training = False)
print(accuracy_NN(pred, y_t))
print(pred)

tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(0.85714287, shape=(), dtype=float32)
tf.Tensor(
[[1.0000000e+00 4.0674752e-08]
 [9.9966788e-01 3.3207354e-04]
 [1.0000000e+00 2.8410946e-15]
 [1.0000000e+00 7.2440810e-11]
 [1.0000000e+00 1.7098918e-08]
 [9.9999988e-01 9.1995268e-08]
 [9.9999726e-01 2.7054641e-06]
 [9.9999857e-01 1.4125483e-06]
 [9.9934870e-01 6.5131544e-04]
 [7.2548020e-04 9.9927455e-01]
 [2.7224720e-01 7.2775280e-01]
 [5.0495164e-03 9.9495053e-01]
 [1.0000000e+00 8.2555345e-11]
 [1.0000000e+00 2.4224992e-08]], shape=(14, 2), dtype=float32)


In [6]:
new_model = tf.keras.models.load_model('test_NN_1.0')
new_model.summary()
print(new_model.trainable_variables)

x = np.array(X, np.float32)
pred = new_model(x, is_training = False)
print(accuracy_NN(pred, Y))
T = pd.read_csv("test_data.csv", index_col = False)
x_t = T.drop('Class', axis = 1)
x_t = np.array(x_t, np.float32)
y_t = T['Class']
pred = new_model(x_t, is_training = False)
print(accuracy_NN(pred, y_t))

Model: "neural__network"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  13056     
_________________________________________________________________
dense_1 (Dense)              multiple                  32896     
_________________________________________________________________
dense_2 (Dense)              multiple                  258       
Total params: 46,210
Trainable params: 46,210
Non-trainable params: 0
_________________________________________________________________
[<tf.Variable 'neural__network/dense/kernel:0' shape=(50, 256) dtype=float32, numpy=
array([[-0.04386537, -0.13827033, -0.09660842, ..., -0.0846169 ,
        -0.012336  , -0.02081989],
       [-0.05211613, -0.12310857, -0.04572839, ...,  0.02974619,
         0.03060456,  0.09097918],
       [-0.01048291, -0.12628607, -0.10894922, ...,  0.02029113,
         0.046785  , -0.049

ValueError: could not convert string to float: '23/3/2021'

In [ ]:
dense_weight = np.asarray(neural_network.trainable_variables[0])
dense_bias = np.asarray(neural_network.trainable_variables[1])
dense1_weight = np.asarray(neural_network.trainable_variables[2])
dense1_bias = np.asarray(neural_network.trainable_variables[3])
dense2_weight = np.asarray(neural_network.trainable_variables[4])
dense2_bias = np.asarray(neural_network.trainable_variables[5])
# np.savetxt('trainable_variables/dense_weight_50x256.csv', np.transpose(dense_weight), delimiter=',')
# np.savetxt('trainable_variables/dense_bias_256.csv', np.transpose(dense_bias), delimiter=',')
# np.savetxt('trainable_variables/dense1_weight_256x128.csv', np.transpose(dense1_weight), delimiter=',')

In [ ]:
dense_weight = np.asarray(neural_network.trainable_variables[0])
dense_bias = np.asarray(neural_network.trainable_variables[1])
dense1_weight = np.asarray(neural_network.trainable_variables[2])
dense1_bias = np.asarray(neural_network.trainable_variables[3])
dense2_weight = np.asarray(neural_network.trainable_variables[4])
dense2_bias = np.asarray(neural_network.trainable_variables[5])
# np.savetxt('trainable_variables/dense_weight_50x256.csv', np.transpose(dense_weight), delimiter=',')
# np.savetxt('trainable_variables/dense_bias_256.csv', np.transpose(dense_bias), delimiter=',')
# np.savetxt('trainable_variables/dense1_weight_256x128.csv', np.transpose(dense1_weight), delimiter=',')
# np.savetxt('trainable_variables/dense1_bias_128.csv', np.transpose(dense1_bias), delimiter=',')
# np.savetxt('trainable_variables/dense2_weight_128x2.csv', np.transpose(dense2_weight), delimiter=',')
# np.savetxt('trainable_variables/dense2_bias_2.csv', np.transpose(dense2_bias), delimiter=',')
np.savetxt('trainable_variables/dense_weight_50x256.csv', dense_weight, delimiter=',')
np.savetxt('trainable_variables/dense_bias_256.csv', dense_bias, delimiter=',')
np.savetxt('trainable_variables/dense1_weight_256x128.csv', dense1_weight, delimiter=',')
np.savetxt('trainable_variables/dense1_bias_128.csv', dense1_bias, delimiter=',')
np.savetxt('trainable_variables/dense2_weight_128x2.csv', dense2_weight, delimiter=',')
np.savetxt('trainable_variables/dense2_bias_2.csv', dense2_bias, delimiter=',')